In [47]:
# Dependencies
import pandas as pd
import re
import os

In [5]:
!ls -lrt ../Data

total 7848
-rw-r--r--@ 1 bincynarath  staff   107939 Aug 14 00:08 Homicide.csv
-rw-r--r--@ 1 bincynarath  staff     1254 Aug 14 14:49 MCI_Metadata.csv
-rw-r--r--@ 1 bincynarath  staff  3705175 Aug 14 15:02 community_assets.csv
-rw-r--r--@ 1 bincynarath  staff   192530 Aug 15 11:32 Toronto_FSA.csv


In [9]:
# Store filepath in a variable
file_one = "../Data/Toronto_FSA.csv"

In [10]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
file_one_df = pd.read_csv(file_one, encoding="ISO-8859-1")

In [11]:
# Show just the header
file_one_df.head()

,Prov/Terr,FSA,Total,Total Income,Average Income,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999",...,"$45,000 to $49,999","$50,000 to $54,999","$55,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 to $149,999","$150,000 to $249,999","$250,000 and over"
0,10,A0A,40150,1.673931e+09,41691.93026,2950,2700,3880,5220,3220,...,1600,1320,1210,1700,1380,1150,960,2240,880,170
1,10,A0B,17820,7.141520e+08,40075.86981,1180,1150,1790,2570,1650,...,710,580,480,710,520,420,340,960,380,60
2,10,A0C,11180,3.787120e+08,33874.06082,780,720,1220,1870,1180,...,440,300,270,390,230,200,180,340,110,10
3,10,A0E,19860,8.285010e+08,41717.06949,1520,1390,2070,2850,1850,...,610,460,450,710,610,580,530,1520,460,60
4,10,A0G,31290,1.089500e+09,34819.43113,2510,1960,3840,5040,2890,...,1110,850,670,1060,780,690,510,1230,340,70


In [12]:
file_one_df = file_one_df.reindex(sorted(file_one_df.columns), axis=1)

In [13]:
# Show a single column
file_one_df.head()

,"$10,000 to $14,999","$100,000 to $149,999","$15,000 to $19,999","$150,000 to $249,999","$20,000 to $24,999","$25,000 to $29,999","$250,000 and over","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999",...,"$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999",Average Income,FSA,Prov/Terr,Total,Total Income,"Under $5,000"
0,3880,2240,5220,880,3220,2770,170,2550,2190,2090,...,1700,1380,1150,960,41691.93026,A0A,10,40150,1.673931e+09,2950
1,1790,960,2570,380,1650,1370,60,1130,960,860,...,710,520,420,340,40075.86981,A0B,10,17820,7.141520e+08,1180
2,1220,340,1870,110,1180,940,10,840,610,550,...,390,230,200,180,33874.06082,A0C,10,11180,3.787120e+08,780
3,2070,1520,2850,460,1850,1400,60,1140,880,800,...,710,610,580,530,41717.06949,A0E,10,19860,8.285010e+08,1520
4,3840,1230,5040,340,2890,2380,70,2180,1740,1450,...,1060,780,690,510,34819.43113,A0G,10,31290,1.089500e+09,2510


In [14]:
#df.sort_values(by=['FSA'])
postal_code = file_one_df[["FSA", "Average Income"]]
postal_code

,FSA,Average Income
0,A0A,41691.93026
1,A0B,40075.86981
2,A0C,33874.06082
3,A0E,41717.06949
4,A0G,34819.43113
...,...,...
1659,X0A,53338.66667
1660,X0B,41097.6378
1661,X0C,45089.63211
1662,NaN,48777.51901
